<a href="https://colab.research.google.com/github/jennif8r/notebookLM/blob/main/NotebookLm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install openai-whisper

In [11]:
!pip install --upgrade pinecone pinecone-client

  Using cached pinecone_client-5.0.1-py3-none-any.whl.metadata (19 kB)
INFO: pip is looking at multiple versions of pinecone-client to determine which version is compatible with other requirements. This could take a while.
  Using cached pinecone_client-5.0.0-py3-none-any.whl.metadata (19 kB)


In [12]:
!sudo apt update
!sudo apt install ffmpeg


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,741 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [3,453 kB]
Fetched 6,451 kB in 6s (1,162 kB/s)
Reading package lists... Done
Building dependency tree... D

In [13]:
!pip install --upgrade unstructured


In [14]:
!pip install pdfminer.six==20220524

  Using cached pdfminer.six-20220524-py3-none-any.whl.metadata (3.7 kB)
Using cached pdfminer.six-20220524-py3-none-any.whl (5.6 MB)
  Attempting uninstall: pdfminer.six
    Found existing installation: pdfminer.six 20231228
    Uninstalling pdfminer.six-20231228:
      Successfully uninstalled pdfminer.six-20231228
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pdfplumber 0.11.4 requires pdfminer.six==20231228, but you have pdfminer-six 20220524 which is incompatible.


In [15]:
!pip install pi-heif

In [16]:
!pip install unstructured-inference


  Using cached pdfminer.six-20231228-py3-none-any.whl.metadata (4.2 kB)
Using cached pdfminer.six-20231228-py3-none-any.whl (5.6 MB)
  Attempting uninstall: pdfminer.six
    Found existing installation: pdfminer.six 20220524
    Uninstalling pdfminer.six-20220524:
      Successfully uninstalled pdfminer.six-20220524


In [17]:
!apt-get update
!apt-get install -y poppler-utils


Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [18]:
#esse aqui funciona o OCR muito importante
! pip install "unstructured[all-docs]" pillow pydantic lxml pillow matplotlib


In [ ]:
import pkg_resources

packages = [
    "openai-whisper",
    "pinecone",
    "pinecone-client",
    "unstructured",
    "pdfminer.six",
    "pi-heif",
    "unstructured-inference",
    "pillow",
    "pydantic",
    "lxml",
    "matplotlib"
]

for pkg in packages:
    try:
        version = pkg_resources.get_distribution(pkg).version
        print(f"{pkg}: {version}")
    except pkg_resources.DistributionNotFound:
        print(f"{pkg} não encontrado")


openai-whisper: 20240930
pinecone: 5.4.1
pinecone-client: 4.1.2
unstructured: 0.16.9
pdfminer.six: 20231228
pi-heif: 0.21.0
unstructured-inference: 0.8.1
pillow: 11.0.0
pydantic: 2.9.2
lxml: 5.3.0
matplotlib: 3.8.0


#Sem Front
Preciso melhorar algumas coisas como:
- Os chunks ser com overlap e ir até o ponto.
- os metadados adicionar metadados mais relevantes.


In [41]:
from unstructured.partition.pdf import partition_pdf
from unstructured.partition.text import partition_text
import whisper
import os

def extract_text_from_file(file_path, file_type):
    if file_type == 'pdf':
        # Usa apenas o modo "fast" para evitar OCR
        elements = partition_pdf(filename=file_path, strategy="fast")
    elif file_type == 'txt':
        elements = partition_text(filename=file_path)
    else:
        raise ValueError("Tipo de arquivo não suportado.")

    text = "\n".join([element.text for element in elements if element.text])
    return text

def transcribe_audio(file_path):
    model = whisper.load_model("base")
    result = model.transcribe(file_path)
    return result['text']

def create_chunks(text, chunk_size=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append({
            'text': chunk,
            'metadata': {
                'length': len(chunk),
                'start_word': i,
                'end_word': min(i + chunk_size, len(words))
            }
        })
    return chunks

def process_file(file_path, file_type):
    if not os.path.isfile(file_path):
        print("Arquivo não encontrado.")
        return

    if file_type in ['pdf', 'txt']:
        text = extract_text_from_file(file_path, file_type)
    elif file_type == 'mp3':
        text = transcribe_audio(file_path)
    else:
        print(f"Tipo de arquivo '{file_type}' não suportado.")
        return

    chunks = create_chunks(text)

    print("Chunks gerados:")
    if len(chunks) == 0:
        print("Nenhum chunk gerado. O arquivo pode não conter texto extraível.")
    for ch in chunks:
        print(ch)



file_path = "/content/NTC.pdf"  # Substitua pelo caminho do seu arquivo
file_type = "pdf"  # Pode ser "pdf", "txt" ou "mp3"

process_file(file_path, file_type)


Chunks gerados:
{'text': 'JULHO/2014 NTC 855 830 Afastamentos mínimos na estrutura Montagem de Redes de Distribuição Isolada de Média Tensão SEE/DPRD Página 1 de 2 NTC 855 830 Afastamentos mínimos na estrutura Montagem de Redes de Distribuição Isolada de Média Tensão Afastamentos mínimos(cm) K ≤ 250 K ≥ 250 D F D F', 'metadata': {'length': 290, 'start_word': 0, 'end_word': 50}}
{'text': '35 15 50 20 OBS: 1. Medidas em centímetros 2. Aumentar a cota em 5cm quando utilizar a outra face do poste 3. Esta distância poderá ser alterada para atendimento ao “CASO 4 e ao CASO 6 da NTC 856004-RDA Primário-Afastamentos Mínimos entre condutroes e solo, desde que mantidos os', 'metadata': {'length': 279, 'start_word': 50, 'end_word': 100}}
{'text': 'demais afastamentos na estrutura. 4. Para pontos energizados não isolados, devem ser respeitados as afastamentos definidos na NTC 856 000–RDA Afastamentos mínimos na estrutura. JULHO/2014 SEE/DPRD Página 2 de 2', 'metadata': {'length': 210, 'start_word'

In [42]:
# Variável para armazenar os chunks como lista de dicionários
def get_chunks_as_list(file_path, file_type):
    if not os.path.isfile(file_path):
        print("Arquivo não encontrado.")
        return []

    if file_type in ['pdf', 'txt']:
        text = extract_text_from_file(file_path, file_type)
    elif file_type == 'mp3':
        text = transcribe_audio(file_path)
    else:
        print(f"Tipo de arquivo '{file_type}' não suportado.")
        return []

    chunks = create_chunks(text)

    if len(chunks) == 0:
        print("Nenhum chunk gerado. O arquivo pode não conter texto extraível.")
    return chunks


# Chamando a função para obter os chunks
data = get_chunks_as_list(file_path, file_type)

# Exibindo a lista de dicionários
print("Lista de dicionários dos chunks:")
for chunk in chunks_list:
    print(chunk)


Lista de dicionários dos chunks:
{'text': 'JULHO/2014 NTC 855 830 Afastamentos mínimos na estrutura Montagem de Redes de Distribuição Isolada de Média Tensão SEE/DPRD Página 1 de 2 NTC 855 830 Afastamentos mínimos na estrutura Montagem de Redes de Distribuição Isolada de Média Tensão Afastamentos mínimos(cm) K ≤ 250 K ≥ 250 D F D F', 'metadata': {'length': 290, 'start_word': 0, 'end_word': 50}}
{'text': '35 15 50 20 OBS: 1. Medidas em centímetros 2. Aumentar a cota em 5cm quando utilizar a outra face do poste 3. Esta distância poderá ser alterada para atendimento ao “CASO 4 e ao CASO 6 da NTC 856004-RDA Primário-Afastamentos Mínimos entre condutroes e solo, desde que mantidos os', 'metadata': {'length': 279, 'start_word': 50, 'end_word': 100}}
{'text': 'demais afastamentos na estrutura. 4. Para pontos energizados não isolados, devem ser respeitados as afastamentos definidos na NTC 856 000–RDA Afastamentos mínimos na estrutura. JULHO/2014 SEE/DPRD Página 2 de 2', 'metadata': {'length': 

#Criar um banco vetorial no Pinecone

Nesse momento iremos criar o banco vetorial do pinecone para subir os embedding

In [20]:
pip install pinecone-client


In [43]:
from pinecone import ServerlessSpec
from pinecone import Pinecone
import os
from google.colab import userdata
api_key = userdata.get('pinecone_key')


# configure client
pc = Pinecone(api_key=api_key)
cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

index_name = 'test'

if index_name not in pc.list_indexes().names():

    pc.create_index(
        index_name,
        dimension=1024,
        metric='cosine',
        spec=spec
    )
# connect to index
index = pc.Index(index_name)
# view index stats
index.describe_index_stats()



{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [44]:
# Gera os embeddings usando o texto de cada chunk em 'data'
embeddings = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[d['text'] for d in data],
    parameters={"input_type": "passage", "truncate": "END"}
)

# Combina os embeddings com os metadados, incluindo um ID único
data = [
    {
        'id': i,  # Adiciona um ID único para cada chunk
        'embedding': embedding,  # O embedding gerado
        'text': data[i]['text'],  # Texto original do chunk
        'metadata': data[i]['metadata']  # Metadados do chunk
    }
    for i, embedding in enumerate(embeddings)
]

# Exibe os resultados
from pprint import pprint
pprint(data)


[{'embedding': {'vector_type': dense, 'values': [0.030487060546875, -0.0016498565673828125, ..., -0.03240966796875, 0.003559112548828125]},
  'id': 0,
  'metadata': {'end_word': 50, 'length': 290, 'start_word': 0},
  'text': 'JULHO/2014 NTC 855 830 Afastamentos mínimos na estrutura Montagem '
          'de Redes de Distribuição Isolada de Média Tensão SEE/DPRD Página 1 '
          'de 2 NTC 855 830 Afastamentos mínimos na estrutura Montagem de '
          'Redes de Distribuição Isolada de Média Tensão Afastamentos '
          'mínimos(cm) K ≤ 250 K ≥ 250 D F D F'},
 {'embedding': {'vector_type': dense, 'values': [0.01514434814453125, 0.0031490325927734375, ..., -0.00601959228515625, -0.0002791881561279297]},
  'id': 1,
  'metadata': {'end_word': 100, 'length': 279, 'start_word': 50},
  'text': '35 15 50 20 OBS: 1. Medidas em centímetros 2. Aumentar a cota em '
          '5cm quando utilizar a outra face do poste 3. Esta distância poderá '
          'ser alterada para atendimento ao “CA

##subindo para o pinecone

In [52]:
vectors = []
for d, e in zip(data, embeddings):
    full_metadata = d['metadata']  # Metadados existentes
    full_metadata['text'] = d['text']  # Adiciona o texto ao metadado
    vectors.append({
        "id": str(d['id']),
        "values": e['values'],
        "metadata": full_metadata
    })

index.upsert(
    vectors=vectors,
    namespace= index_name
)


{'upserted_count': 3}

SLA O Q È

In [ ]:
from unstructured.partition.pdf import partition_pdf
from unstructured.partition.text import partition_text
import whisper
import os
from pinecone import Pinecone, ServerlessSpec
import uuid

# Função para extrair texto do arquivo
def extract_text_from_file(file_path, file_type):
    if file_type == 'pdf':
        elements = partition_pdf(filename=file_path, strategy="fast")
    elif file_type == 'txt':
        elements = partition_text(filename=file_path)
    else:
        raise ValueError("Tipo de arquivo não suportado.")
    return "\n".join([element.text for element in elements if element.text])

# Função para transcrever áudio
def transcribe_audio(file_path):
    model = whisper.load_model("base")
    result = model.transcribe(file_path)
    return result['text']

# Função para criar chunks de texto
def create_chunks(text, chunk_size=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append({
            'text': chunk,
            'metadata': {
                'length': len(chunk),
                'start_word': i,
                'end_word': min(i + chunk_size, len(words))
            }
        })
    return chunks

# Processa o arquivo e retorna os chunks
def process_file(file_path, file_type):
    if not os.path.isfile(file_path):
        raise FileNotFoundError("Arquivo não encontrado.")

    if file_type in ['pdf', 'txt']:
        text = extract_text_from_file(file_path, file_type)
    elif file_type == 'mp3':
        text = transcribe_audio(file_path)
    else:
        raise ValueError(f"Tipo de arquivo '{file_type}' não suportado.")

    return create_chunks(text)

# Configura o Pinecone
def setup_pinecone(api_key, index_name, dimension=768):
    pc = Pinecone(api_key=api_key)
    cloud = "aws"
    region = "us-east-1"
    spec = ServerlessSpec(cloud=cloud, region=region)

    if index_name not in pc.list_indexes().names():
        pc.create_index(
            name=index_name,
            dimension=dimension,
            metric="cosine",
            spec=spec
        )
        print(f"Índice '{index_name}' criado.")
    else:
        print(f"Índice '{index_name}' já existe. Usando o índice existente.")

    return pc.Index(index_name)

# Principal: processa, cria embeddings e sobe no Pinecone
def main():
    # Solicita as informações do usuário
    api_key = input("Insira sua chave de API do Pinecone: ")
    index_name = input("Insira o nome do índice: ")
    file_path = input("Insira o caminho do arquivo para processar: ")
    file_type = file_path.split('.')[-1].lower()  # Determina o tipo do arquivo pelo sufixo

    # Processa o arquivo
    chunks = process_file(file_path, file_type)
    print(f"{len(chunks)} chunks gerados.")

    # Configura Pinecone
    index = setup_pinecone(api_key, index_name)

    # Gera embeddings diretamente no Pinecone
    texts = [chunk['text'] for chunk in chunks]
    embeddings = index._transport.inference.embed(
        model="multilingual-e5-large",
        inputs=texts,
        parameters={"input_type": "passage", "truncate": "END"}
    )

    # Prepara os registros para o Pinecone
    records = []
    for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
        records.append({
            "id": str(uuid.uuid4()),
            "values": embedding,  # Vetores já estão no formato correto
            "metadata": {
                "filename": os.path.basename(file_path),  # Nome do arquivo
                **chunk['metadata']  # Metadados adicionais
            }
        })

    # Insere os dados no Pinecone
    index.upsert(vectors=records)
    print(f"Dados inseridos no índice '{index_name}' com sucesso.")

# Executa o script principal
if __name__ == "__main__":
    main()


Insira sua chave de API do Pinecone: pcsk_4v6tXf_4kzQ3sdX8HhMJBqQS9jiGpyDyPbXe3opJot8aGaSYyNgQjyJRrSxwZt4Mz8TAhc
Insira o nome do índice: testau
Insira o caminho do arquivo para processar: /content/Normas (1).pdf
11 chunks gerados.
Índice 'testau' criado.


AttributeError: 'Index' object has no attribute '_transport'

#Front

In [ ]:
pip install gradio


In [ ]:
import os
import gradio as gr
import whisper
from unstructured.partition.pdf import partition_pdf
from unstructured.partition.text import partition_text

def guess_file_type(filename):
    ext = os.path.splitext(filename)[1].lower()
    if ext == ".pdf":
        return "pdf"
    elif ext == ".txt":
        return "txt"
    elif ext in [".mp3", ".wav", ".m4a", ".ogg"]:
        return "mp3"
    return None

def extract_text_from_file(file_path, file_type):
    if file_type == 'pdf':
        elements = partition_pdf(filename=file_path, strategy="fast")
    elif file_type == 'txt':
        elements = partition_text(filename=file_path)
    else:
        raise ValueError("Tipo de arquivo não suportado.")

    text = "\n".join([element.text for element in elements if element.text])
    return text

def transcribe_audio(file_path):
    model = whisper.load_model("base")
    result = model.transcribe(file_path)
    return result['text']

def create_chunks(text, chunk_size=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append({
            'text': chunk,
            'metadata': {
                'length': len(chunk),
                'start_word': i,
                'end_word': min(i + chunk_size, len(words))
            }
        })
    return chunks

def process_file(file_path):
    file_type = guess_file_type(file_path)
    if not file_type:
        return "Tipo de arquivo não suportado ou desconhecido."

    if not os.path.isfile(file_path):
        return "Arquivo não encontrado."

    if file_type in ['pdf', 'txt']:
        text = extract_text_from_file(file_path, file_type)
    elif file_type == 'mp3':
        text = transcribe_audio(file_path)
    else:
        return f"Tipo de arquivo '{file_type}' não suportado."

    chunks = create_chunks(text)
    if len(chunks) == 0:
        return "Nenhum chunk gerado. O arquivo pode não conter texto extraível."

    # Formata os chunks para exibição
    output = "Chunks gerados:\n\n"
    for ch in chunks:
        output += f"Texto: {ch['text']}\nMetadados: {ch['metadata']}\n\n"
    return output

def handle_file_upload(uploaded_file):
    if uploaded_file is None:
        return "Nenhum arquivo enviado."
    return process_file(uploaded_file.name)

with gr.Blocks() as demo:
    gr.Markdown("# Extração de Texto / Transcrição via Gradio")
    with gr.Row():
        file_input = gr.File(label="Envie seu arquivo (PDF, TXT, MP3, WAV, ...)")
    output = gr.Textbox(label="Resultado", lines=20)
    process_button = gr.Button("Processar Arquivo")

    process_button.click(fn=handle_file_upload, inputs=file_input, outputs=output)

if __name__ == "__main__":
    demo.launch()
